In [ ]:
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# Load and preprocess data (example)
def load_audio(file_path):
    signal, sr = librosa.load(file_path, sr=22050)
    return signal, sr

def extract_features(signal, sr):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

# Example dataset


# Path to the directory
directory_path = '/content/drive/MyDrive/crying'

# Check if the directory exists
if os.path.exists(directory_path):
    # List all files in the directory with their full paths
    crying_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
else:
    print("The directory does not exist.")

directory_path2 = '/content/drive/MyDrive/background sounds'

if os.path.exists(directory_path2):
    # List all files in the directory with their full paths
    non_crying_files = [os.path.join(directory_path2, f) for f in os.listdir(directory_path2) if os.path.isfile(os.path.join(directory_path2, f))]
else:
    print("The directory does not exist.")

X = []
y = []

for file in crying_files:
    signal, sr = load_audio(file)
    features = extract_features(signal, sr)
    X.append(features)
    y.append(1)  # Crying

for file in non_crying_files:
    signal, sr = load_audio(file)
    features = extract_features(signal, sr)
    X.append(features)
    y.append(0)  # Not crying

X = np.array(X)
y = np.array(y)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model definition
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Save the model
model.save('crying_detection_model.h5')

# Real-time detection
def predict_crying(audio_segment):
    features = extract_features(audio_segment, sr=22050)
    features = features.reshape(1, -1)
    prediction = model.predict(features)
    return prediction > 0.5


In [ ]:
import numpy as np
import librosa
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/crying_detection_model.h5')

# Function to preprocess the audio and extract features
def load_and_preprocess_audio(file_path):
    signal, sr = librosa.load(file_path, sr=22050)
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

# Function to predict if the baby is crying
def predict_crying(file_path):
    features = load_and_preprocess_audio(file_path)
    features = features.reshape(1, -1)
    prediction = model.predict(features)
    return prediction > 0.5

# Test audio file paths
test_files = ['/content/drive/MyDrive/background sounds/Recording (2).m4a']

# Predict and print the results
for file in test_files:
    is_crying = predict_crying(file)
    print(f"File: {file}, Crying: {'Yes' if is_crying else 'No'}")

